In [ ]:
import os
import gc
import pyperclip as clip
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/gdrive/projects/programming/automation")
from time import sleep
import sistemas
import sei
import functions
from page import *
import selenium.webdriver as webdriver

% reload_ext autoreload
% autoreload 2

In [ ]:
PATH = os.path.abspath(r"C:/Users/rsilva/Desktop/SEI/")
print(PATH)               

In [ ]:
keys = ('Nome/Razão Social', 'Logradouro', 'Número', 'Complemento', 'Bairro', 'Cep', 'Município', 'UF', 'Número Fistel')

In [ ]:
def string_endereço(dados):
    
    d = {}
    
    s = 'A(o)<br><br>' 
    
    s += dados["Nome/Razão Social"].title()
    
    s += '<br>' + dados["Logradouro"].title() + ", " + dados["Número"] + " " 
    
    s += dados["Complemento"].title() + " " 
    
    s += dados["Bairro"].title() + '<br>' 
    
    s += "CEP: " + dados["Cep"] + " - " + dados["Município"].title() + " - " + dados["UF"] 
            
    s += "<br><br>" + "<b>FISTEL: " + dados["Número Fistel"] + "</b>"
    
    d["A"] = s
    
    return d

In [ ]:
cpf = '351.987.128-92'
proc = r'53504.006105/2018-03'

# Scpx - Manipulação de Cadastro

In [ ]:
driver = webdriver.Ie()
scpx = sistemas.Scpx()

### Consulta

In [ ]:
scpx.consulta(cpf)

### Serviço

In [ ]:
scpx.servico_incluir(cpf, num_processo=proc, silent=True)

In [ ]:
#scpx.servico_excluir(cpf, num_processo=proc)

## Estação

In [ ]:
scpx.prorrogar_rf(cpf)

In [ ]:
scpx.prorrogar_estacao(cpf)

In [ ]:
scpx.incluir_estacao(cpf, 'Fixa', 'PX2J1609', sequencial='002')

In [ ]:
scpx.incluir_estacao(cpf, 'Móvel', 'PX2R1986', sequencial='001')

In [ ]:
scpx.movimento_transferir(cpf, 'B', 'E', proc=proc)

In [ ]:
scpx.movimento_transferir(cpf, 'A', 'G', proc=proc)

In [ ]:
scpx.movimento_cancelar(cpf)

In [ ]:
scpx.licenciar_estacao(cpf, silent=True)

In [ ]:
scpx.licenciar_estacao(cpf, ppdess=False, silent=True)

In [ ]:
dados = scpx.extrai_cadastro(cpf)

## Licença

In [ ]:
scpx.imprimir_licenca(cpf)

# SEI - Manipulação de Processo

## Autenticação

In [ ]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [ ]:
sei_ = sei.login_sei(browser, usr, pwd)

In [ ]:
sei_ = sei.Sei(sei_)

In [ ]:
sei_.itera_processos()

In [ ]:
p = sei_.go_to_processo('53504.011700/2017-71')

## Doc Externo

In [ ]:
files = sorted(os.listdir(PATH), reverse=True)
file = os.path.join(PATH, files[0])

In [ ]:
p.incluir_doc_externo('Relação', file, arvore="de Entidades Não Prorrogadas")

In [ ]:
p.incluir_oficio('RC_OF_Sem_Contexto_PPDESS', string_endereço(dados))

## Controles

In [ ]:
p.edita_marcador(tipo="RC - Aguardando AR/Resposta", content='Até 30/08/2018')

In [ ]:
p.edita_marcador()

In [ ]:
p.edita_postit()

In [ ]:
p.excluir_acomp_especial()

In [ ]:
p.concluir_processo()

## Testes

In [ ]:
tipo = "Outorga: Rádio do Cidadão"
atribuicao = 'rodrigoca'

In [ ]:
processos = [p for p in sei_.get_processos().values() if p['atribuicao'] == 'rodrigoca']

In [ ]:
mm = [p for p in processos if p['tipo'] == tipo]

In [ ]:
for p in mm:
    print(p.get('marcador'))

In [ ]:
concluir = list()

In [ ]:
for processo in processos:
    
    if 'Licença' in str(processo.get('marcador')):
        
        p = sei_.go_to_processo(processo['numero'])
        
        i = input()
        
        if i == '1': 
            
            concluir.append(processo)
            
            
        gc.collect()

In [ ]:
for processo in concluir:
    
    p = sei_.go_to_processo(processo['numero'])
    
    
    sleep(2)
    
    try:    
        p.edita_marcador()    
    
        p.edita_postit()
    
        p.excluir_acomp_especial()
        
        p.concluir_processo() 
            
    except:
            
            next

In [ ]:
#dados = scpx.extrai_cadastro(dado['interessado'], tipo_id='id_nome', timeout=5)


In [ ]:
for num, dado in tupla[30:]:
        
    p = sei_.go_to_processo(num)
    
    i = input()
    
    if i =='y':
        
        try:
            
            dados = scpx.extrai_cadastro(dado['interessado'], tipo_id='id_nome', timeout=5)
                               
            p.incluir_oficio('RC_OF_Sem_Contexto_Licença', string_endereço(dados))
            
            p.edita_marcador(tipo='RC - Aguardando AR/Resposta', content='Até 31/07/2018')        

            
        except:
            
            next            
            
    if i == 'c':
        
        p.edita_marcador()
        p.edita_postit()
        p.excluir_acomp_especial()
        p.concluir_processo() 

## Outros

In [ ]:
from bs4 import BeautifulSoup as soup
html = soup(scpx.driver.page_source, 'lxml')

In [ ]:
dict_ = {}
for tr in html.find_all('tr'):
    for td in tr.find_all('td', string=True):
        key = td.text.strip(" :")
        value = td.find_next_sibling('td')            
        if value and key not in dict_ and value.text.strip() != "":
            dict_[key] = value.text.strip()
            print("{0} {1}".format(key, value.text.strip())) 


In [ ]:
dict_ = {}
for tr in html.find_all('tr'):
    for label in tr.find_all('label', string=True):
        print("{0} {1}".format(label.attrs.get('id'), label.text.strip()))                           

In [ ]:
from bs4 import NavigableString
def next_is_string(tag):
    return isinstance(tag.next_element, NavigableString)             

# Boleto

In [7]:
boleto = sistemas.Boleto(driver)

In [8]:
fistel = '80113639805'
cpf = "31250732867"

In [19]:
boleto.imprime_boleto(fistel, tipo_id='id_fistel')

A espera pela nova janela não funcionou!
